###### imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib import transforms
import scipy
import scipy.stats as st
from SequenceGenerator import MultiSequenceGenerator
from scipy.integrate import odeint
import networkx as nx
import random
import matplotlib.colors as mcolors
import matplotlib.cm as cm

#  Bose-Einstein

A fitness distribution that leads to a Bose-Einstein condensation is:
$${\rho}({\eta}) = (1 + {\zeta}) (1 - {\eta})^{\zeta}$$

## distribution

### implementation

In [ ]:
def distribution(eta, zeta): # eta is the input, like x
    return (1.0 + zeta) * np.power(1.0 - eta, zeta)

### zeta parameter effect

###### parameters

In [ ]:
interval = (0.0, 1.0)
sample_number = 100
zeta_list = [0.1, 10.0]

###### generating values

In [ ]:
x = np.linspace(interval[0], interval[1], sample_number)

y = np.array([distribution(x, zeta) for zeta in zeta_list])

##### plotting results

###### seaborn

In [ ]:
for i in range(len(zeta_list)):
    sns.displot(y[i], kde=True, stat='probability')
    plt.title(f'zeta= {zeta_list[i]}')
    plt.legend(loc='best')
    plt.show()

###### simple plot

In [ ]:
for i in range(len(zeta_list)):
    plt.plot(x, y[i], label=f'zeta= {zeta_list[i]}')
plt.title('zeta effect')
plt.legend(loc='best')
plt.show()

###### semilog plot

In [ ]:
for i in range(len(zeta_list)):
    plt.plot(x, y[i], label=f'zeta= {zeta_list[i]}')
plt.title('zeta effect')
plt.legend(loc='best')
plt.semilogy()
plt.show()

###### log-log plot

In [ ]:
for i in range(len(zeta_list)):
    plt.plot(x, y[i], label=f'zeta= {zeta_list[i]}')
plt.title('zeta effect')
plt.legend(loc='best')
plt.xscale('log')
plt.yscale('log')
plt.show()

## Bose-Einstein implementation

### implementation

In [ ]:
class BoseEinstein():
    def draw_graph_for_time_slot(self, step_number):
        print(f'step number: {step_number}')
        fig = plt.figure(figsize=(50, 50))
        degrees = np.array([self.graph.degree(n) for n in self.graph.nodes()])
        node_size = degrees*100
        
        pos=nx.spring_layout(self.graph)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        plt.title(f'step number: {step_number}')
        plt.show()
        
    
    def append_nodes(self):
        time_intervals = int(self.step_number/self.time_slot_number)
        time_slots = list(range(time_intervals, self.step_number+1, time_intervals))
        
        for new_node in range(self.initial_node_number+1, self.step_number+self.initial_node_number+1):
            degrees = [val for (node, val) in sorted(self.graph.degree(), key=lambda pair: pair[0])]
            summation = np.sum([self.fitness[i]*degrees[i] for i in range(len(degrees))])
            probabilities = [(self.fitness[i]*degrees[i])/summation for i in range(len(degrees))]
            probabilities_cum = np.cumsum(probabilities)
            interval_max = probabilities_cum[len(probabilities_cum) - 1]
            
            self.graph.add_node(new_node)
            
            for _ in range(self.new_node_link):
                random_number = np.random.uniform(0.0, interval_max)
                connected_node = sum(i < random_number for i in probabilities_cum) + 1 # pluse one is because node numbers starts from 1 not zero
                self.graph.add_edge(new_node, connected_node)

            step_number = new_node - (self.initial_node_number+1)
            if (step_number in time_slots):
                self.draw_graph_for_time_slot(step_number)
    
    def __init__(self, initial_node_number=3, step_number=100, new_node_link=3, fitness=[], time_slot_number=10):
        self.initial_node_number=initial_node_number
        self.step_number=step_number
        self.new_node_link=new_node_link
        self.time_slot_number = time_slot_number
        self.fitness = fitness
        self.graph = nx.complete_graph(self.initial_node_number+1)
        self.graph.remove_node(0)
        self.append_nodes()
        self.set_degree_distribution()

    def set_degree_distribution(self):
        self.degree_distribution = np.array([self.graph.degree(n) for n in self.graph.nodes()])
        self.degree_distribution = np.sort(self.degree_distribution)[::-1]

    def draw_degree_distribution(self):
        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution semilog')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.semilogy()
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution log-log')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.xscale('log')
        plt.yscale('log')
        plt.show()

    def draw_graph(self):
        fig = plt.figure(figsize=(50, 50))
        node_size = np.array([self.graph.degree(n) for n in self.graph.nodes()]) * 100
        
        pos=nx.spring_layout(self.graph)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        plt.show()

### simulation

#### simulation for general ${\zeta}$   (limiting to see star-like hub-and-spoke topology)

###### parameters

In [ ]:
initial_node_number=3
step_number=1000
new_node_link=3
zeta = 1000.0

interval = (0.0, 1.0)
sample_number = step_number + initial_node_number

x = np.linspace(interval[0], interval[1], sample_number)

fitness = np.array(distribution(x, zeta))

###### generating values

In [ ]:
g_holder = BoseEinstein(initial_node_number=initial_node_number, step_number=step_number, new_node_link=new_node_link, fitness=fitness)

##### drawing plots

###### degree distribution

In [ ]:
g_holder.draw_degree_distribution()

###### drawing graph

In [ ]:
g_holder.draw_graph()

#### ${\zeta} = 10$

###### parameters

In [ ]:
initial_node_number=3
step_number=1000
new_node_link=3
zeta = 10.0

interval = (0.0, 1.0)
sample_number = step_number + initial_node_number

x = np.linspace(interval[0], interval[1], sample_number)

fitness = np.array(distribution(x, zeta))

###### generating values

In [ ]:
g_holder = BoseEinstein(initial_node_number=initial_node_number, step_number=step_number, new_node_link=new_node_link, fitness=fitness)

##### drawing plots

###### degree distribution

In [ ]:
g_holder.draw_degree_distribution()

###### draw graph

In [ ]:
g_holder.draw_graph()

#### ${\zeta} = 0.1$

###### parameters

In [ ]:
initial_node_number1=3
step_number1=1000
new_node_link1=3
zeta1 = 1.0

interval1 = (0.0, 1.0)
sample_number1 = step_number1 + initial_node_number1

x1 = np.linspace(interval1[0], interval1[1], sample_number1)

fitness1 = np.array(distribution(x1, zeta1))

###### generating values

In [ ]:
g_holder1 = BoseEinstein(initial_node_number=initial_node_number1, step_number=step_number1, new_node_link=new_node_link1, fitness=fitness1)

##### plotting results

###### degree distribution

In [ ]:
g_holder1.draw_degree_distribution()

###### draw graph

In [ ]:
g_holder1.draw_graph()

#### comparison between $\zeta$ = 10.0, 10

###### simple plot

In [ ]:
plt.plot(g_holder.degree_distribution, label='zeta=10.0')
plt.plot(g_holder1.degree_distribution, label='zeta=1.0')
plt.ylabel('degree')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.hist(g_holder.degree_distribution, label='zeta=10.0')
plt.hist(g_holder1.degree_distribution, label='zeta=1.0')
plt.ylabel('degree')
plt.legend(loc='best')
plt.show()

In [ ]:
degree_freq = nx.degree_histogram(g_holder.graph)
degrees = range(len(degree_freq))
plt.plot(degrees, degree_freq,'o-', label='zeta=10.0') 

degree_freq = nx.degree_histogram(g_holder1.graph)
degrees = range(len(degree_freq))
plt.plot(degrees, degree_freq,'o-', label='zeta=1.0') 

plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.legend(loc='best')
plt.show()

###### semilog plot

In [ ]:
plt.plot(g_holder.degree_distribution, label='zeta=10.0')
plt.plot(g_holder1.degree_distribution, label='zeta=1.0')
plt.legend(loc='best')
plt.semilogy()
plt.show()

###### log-log

In [ ]:
plt.plot(g_holder.degree_distribution, label='zeta=10.0')
plt.plot(g_holder1.degree_distribution, label='zeta=1.0')
plt.ylabel('degree')
plt.legend(loc='best')
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
degree_freq = nx.degree_histogram(g_holder.graph)
degrees = range(len(degree_freq))
plt.loglog(degrees, degree_freq,'o-', label='zeta=10.0') 

degree_freq = nx.degree_histogram(g_holder1.graph)
degrees = range(len(degree_freq))
plt.loglog(degrees, degree_freq,'o-', label='zeta=1.0') 

plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.legend(loc='best')
plt.show()

#### comparison for general $\zeta$

###### parameters

In [ ]:
initial_node_number=3
step_number=100
new_node_link=3
zeta_list = [5.0, 2.0, 0.2, 0.1]

interval = (0.0, 1.0)
sample_number = step_number + initial_node_number

x = np.linspace(interval[0], interval[1], sample_number)

fitness_list = [np.array(distribution(x, zeta)) for zeta in zeta_list]

###### generating values

In [ ]:
g_holder_list = [BoseEinstein(initial_node_number=initial_node_number, step_number=step_number, new_node_link=new_node_link, fitness=fitness) for fitness in fitness_list]

###### simple plot

In [ ]:
for i in range(len(zeta_list)):
    plt.plot(g_holder_list[i].degree_distribution, label=f'zeta={zeta_list[i]}')
plt.ylabel('degree')
plt.legend(loc='best')
plt.show()

In [ ]:
for i in range(len(zeta_list)):
    plt.hist(g_holder_list[i].degree_distribution, label=f'zeta={zeta_list[i]}')
plt.legend(loc='best')
plt.ylabel('degree')
plt.show()

In [ ]:
for i in range(len(zeta_list)):
    degree_freq = nx.degree_histogram(g_holder_list[i].graph)
    degrees = range(len(degree_freq))
    plt.plot(degrees, degree_freq,'o-', label=f'zeta={zeta_list[i]}')

plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.legend(loc='best')
plt.show()

###### semilog

In [ ]:
for i in range(len(zeta_list)):
    plt.plot(g_holder_list[i].degree_distribution, label=f'zeta={zeta_list[i]}')
plt.legend(loc='best')
plt.semilogy()
plt.show()

###### log-log plot

In [ ]:
for i in range(len(zeta_list)):
    plt.plot(g_holder_list[i].degree_distribution, label=f'zeta={zeta_list[i]}')
plt.ylabel('degree')
plt.legend(loc='best')
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
for i in range(len(zeta_list)):
    degree_freq = nx.degree_histogram(g_holder_list[i].graph)
    degrees = range(len(degree_freq))
    plt.loglog(degrees, degree_freq,'o-', label=f'zeta={zeta_list[i]}')

plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.legend(loc='best')
plt.show()